# روش چک کردن تمام حالت های ممکن برای رسید به ترکیب مناسب فیچر ها برای هر الگوریتم

برای رسیدن به تمام حالات فیچر ها یعنی هر 511 حالت ممکن از تابع 
poweset
استفاده می کنیم

که خروجی این تابع تمام ترکیب های ممکن این 9 فیچر است


همینطور از تابع
unison_shuffle
 برای شافل کردن داده ها استفاده می کنیم
 
 
 
 
 این روش به اینصورت است که در یک حلقه تک تک 511 حالت به این شیوه چک می شود که 
 اول مدل مورد نظر را انتخاب کرده و پس از آن 40 بار داده ها را شافل کرده و در هر بار با روش 
 k-fold
مقدار میانگین صحت داده های تست و ترین را بدست می آوریم پس اجرا حلقه 40 تایی مقدار صحت ها را در این 40 بار میانگین می گیریم



بعد بیشترین مقدار صحت برای تست و ترین را پیدا کرده و برای هر با بررسی مقادیر بهترین ترکیب را برای اعمال به الگوریتم پیدا می کنیم
  

In [ ]:
import numpy as np
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.feature_selection import SelectKBest, chi2
from sklearn import svm
from sklearn import tree
from sklearn.neighbors import KNeighborsClassifier


def powerset(s):
    x = len(s)
    out=[]
    for i in range(1 << x):
        out.append([s[j] for j in range(x) if (i & (1 << j))])
    return out


def unison_shuffle(a,b):
    inx=np.random.permutation(a.shape[0])
    return a[inx],b[inx]


df=pd.read_csv('dataR2.csv')

df_1=df.drop(['Classification'],axis=1)
process_data=pd.DataFrame()

t=df_1.mean()
r=df_1.std()
df_1=(df_1-t)/r#داده ها را نورمالایز می کنیم


y=df['Classification'].to_numpy()
y=y-1#در انجا خروجی را به این شکل یه صورت صفر و یک در می آوریم

kf=KFold(n_splits=5)


logisticRegression_model=svm.SVC(kernel='rbf',C=5)#این متغیر برابر با مدل است 
#و برای چک کردن هر الگوریتم نیاز است مدل مورد نظر را در اینجا تعریف کنیم



all_labels=['Age','Glucose','BMI','Resistin','Adiponectin','Insulin','HOMA','Leptin','MCP.1']


all_mood=powerset(all_labels)#این لیست شامل تمام ترکیب های 9 فیچر کنار هم است
del all_mood[0]#عضو اول لیست که تهی است را حذف می کنیم

#all_mood=[['Age', 'Glucose', 'BMI', 'Adiponectin'],
  #        ['Resistin', 'Adiponectin', 'Insulin'],
 #        ['Glucose', 'BMI', 'Resistin', 'Adiponectin']]

#samp=all_mood[30:70]

inx=0

total_train=[]
total_test=[]

for k in all_mood:
    
    acc_train_mood=[]
    acc_test_mood=[]
    
    acc_train_mood.clear()
    acc_test_mood.clear()
    
    
    inx=inx+1
    process_data=df_1[k]
    x=process_data.to_numpy()
    y=df['Classification'].to_numpy()
    y=y-1
    
    
    (x,y)=unison_shuffle(x,y)# داده ها بعد از هر مرحله از 40 بار شافل می شوند
    
    
    for l in range(40):
        (x,y)=unison_shuffle(x,y)
        acc_train_logisticRegression=[]
        acc_test_logisticRegression=[]
        
        acc_train_logisticRegression.clear()
        acc_test_logisticRegression.clear()
        
        for train_index, test_index in kf.split(x):
            X_train, X_test = x[train_index], x[test_index]
            y_train, y_test = y[train_index], y[test_index]
            logisticRegression_model.fit(X_train, y_train)
            acc_train_logisticRegression.append(logisticRegression_model.score(X_train, y_train))
            acc_test_logisticRegression.append(logisticRegression_model.score(X_test, y_test))
            
        acc_train_mood.append(sum(acc_train_logisticRegression)/len(acc_train_logisticRegression))
        acc_test_mood.append(sum(acc_test_logisticRegression)/len(acc_test_logisticRegression))
        
        
            
            
    n=sum(acc_train_mood) /len(acc_train_mood)
    total_train.append(n)
    m=sum(acc_test_mood)/len(acc_test_mood)
    total_test.append(m)
    
    
   # print(inx)
   # print(k)
   # print('acc_train='+str(n))
   # print('acc_test='+str(m))    
   # print('###############################################################################')

# در اینجا مقدار بیشینه برای صحت برای حالت های تست و ترین را میابیم

In [66]:
print(max(total_train))
print(max(total_test))

0.95346306685367
0.8131340579710142


In [67]:
total_train.index(max(total_train))


446

In [68]:
total_test.index(max(total_test))

10

In [70]:
print(all_mood[446])
print(total_train[446])
print(total_test[446])

['Age', 'Glucose', 'BMI', 'Resistin', 'Adiponectin', 'Insulin', 'Leptin', 'MCP.1']
0.95346306685367
0.7342119565217392


In [69]:
print(all_mood[10])
print(total_train[10])
print(total_test[10])

['Age', 'Glucose', 'Resistin']
0.8698316970546983
0.8131340579710142
